In [1]:
import open3d
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import tqdm
import copy
import cv2

import utils.registration as registration
import utils.fread as fread
import utils.transform as transform
import utils.grid_search as grid_search
import utils.pointcloud as pointcloud
import utils.functions as functions
import utils.FCGF as FCGF

from time import sleep
from PIL import Image
from utils.config import Config

In [25]:
config = Config(
        sequence_dir="data/raw_data",
        feature_dir="data/features",
        output_dir="data/trajectories/trajectory/FPFH_outlier_removed",
        experiment="exp_12",
        trial="trial_1",
        subject="subject-1",
        sequence="03",
        groundtruth_dir="data/trajectories/groundtruth",
    )
    
config.voxel_size=0.03
config.target_fps=20
config.min_std=0.5

In [40]:
sequence_ts = fread.get_timstamps(config.get_feature_dir(), ext=".secondary.npz")

groundtruth_data = np.load(os.path.join(config.get_groundtruth_dir(), f"{config.get_file_name()}.gtpose.npz"))
groundtruth_t = groundtruth_data["local_t"]

num_frames = len(sequence_ts)

local_pcds = []

for t in tqdm.trange(num_frames):
    feature_file = os.path.join(config.get_feature_dir(), f"{sequence_ts[t]}.secondary.npz")
    pcd = FCGF.get_features(feature_file, config.voxel_size, pcd_only=True)
    local_pcds.append(pcd)

global_pcds = []

for t in tqdm.trange(num_frames):
    feature_file = os.path.join(config.get_feature_dir(), f"{sequence_ts[t]}.global.npz")
    pcd = FCGF.get_features(feature_file, config.voxel_size, pcd_only=True)
    global_pcds.append(pcd)

100%|██████████| 613/613 [00:28<00:00, 21.75it/s]


In [31]:
fragments_gt = []

for t in range(0, num_frames, 10):
    local_temp = copy.deepcopy(local_pcds[t])
    local_temp.transform(groundtruth_t[t])
    fragments_gt.append(local_temp)
    
    if t == 0:
        continue
    
    cpoints = np.asarray(local_temp.points)
    cx, cy = cpoints[:, 0], cpoints[:, 2]

    plt.scatter(gx, gy, s=0.1, c="g")
    plt.scatter(cx, cy, s=0.1, c="r")
    plt.xlim(-4.5, 4)
    plt.ylim(-5.5, 4.5)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.axis("off")
    
    plt.savefig(f"results/presentation/local/{t:03d}.jpeg")
    plt.close()
    
    trajectory = pointcloud.merge_pcds(fragments_gt, 0.03)

    points = np.asarray(trajectory.points)
    gx, gy = points[:, 0], points[:, 2]

In [38]:
img_files = glob.glob("results/presentation/local/*.jpeg")
img_files = sorted(img_files, key=lambda x: int(os.path.basename(x).split(".")[0]))
images = []

for t in range(len(img_files)):
    img = cv2.imread(img_files[t])
    images.append(img)

video = cv2.VideoWriter(f"results/presentation/local/video.avi", cv2.VideoWriter_fourcc(*'XVID'), 10, (432, 288))
for image in tqdm.tqdm(images):
    video.write(image)

video.release()

100%|██████████| 60/60 [00:00<00:00, 3082.20it/s]


In [41]:
for t in range(0, num_frames, 10):
    local_temp = copy.deepcopy(local_pcds[t])
    local_temp.transform(groundtruth_t[t])
    fragments_gt.append(local_temp)
    
    cpoints = np.asarray(local_temp.points)
    cx, cy = cpoints[:, 0], cpoints[:, 2]
    
    gpoints = np.asarray(global_pcds[t].points)
    gx, gy = gpoints[:, 0], gpoints[:, 2]

    plt.scatter(gx, gy, s=0.1, c="b")
    plt.scatter(cx, cy, s=0.1, c="r")
    plt.xlim(-4.5, 4)
    plt.ylim(-5.5, 4.5)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.axis("off")
    
    plt.savefig(f"results/presentation/global/{t:03d}.jpeg")
    plt.close()

In [47]:
img_files = glob.glob("results/presentation/grid_ransac/*.jpeg")
img_files = sorted(img_files, key=lambda x: int(os.path.basename(x).split(".")[0]))
images = []

for t in range(len(img_files)):
    img = cv2.imread(img_files[t])
    images.append(img)

video = cv2.VideoWriter(f"results/presentation/grid_ransac/video.avi", cv2.VideoWriter_fourcc(*'XVID'), 10, (432, 288))
for image in tqdm.tqdm(images):
    video.write(image)

video.release()

100%|██████████| 61/61 [00:00<00:00, 1626.98it/s]


In [44]:
import utils.grid_search_unopt as grid_search

In [46]:
for t in range(0, num_frames, 10):
    source_feature_file = os.path.join(config.get_feature_dir(), f"{sequence_ts[t]}.secondary.npz")
    target_feature_file = os.path.join(config.get_feature_dir(), f"{sequence_ts[t]}.global.npz")

    source, target, reg_result = grid_search.global_registration(source_feature_file, target_feature_file, config.voxel_size, cell_size=2, refine_enabled=True)

    if reg_result is None:
        reg_result = np.identity(4)
    else:
        reg_result = reg_result.transformation

    source.transform(reg_result)

    cpoints = np.asarray(source.points)
    cx, cy = cpoints[:, 0], cpoints[:, 2]

    gpoints = np.asarray(target.points)
    gx, gy = gpoints[:, 0], gpoints[:, 2]

    plt.scatter(gx, gy, s=0.1, c="b")
    plt.scatter(cx, cy, s=0.1, c="r")
    plt.xlim(-4.5, 4)
    plt.ylim(-5.5, 4.5)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.axis("off")

    plt.savefig(f"results/presentation/grid_ransac/{t:03d}.jpeg")
    plt.close()

In [21]:
config = Config(
        sequence_dir="D://Projects/Research/local-registration/data/raw_data",
        feature_dir="D://Projects/Research/local-registration/data/features",
        output_dir="data/trajectories/trajectory/",
        experiment="exp_8",
        trial="trial_1",
        subject="subject-1",
        sequence="02",
        groundtruth_dir="D://Projects/Research/local-registration/data/trajectories/groundtruth",
    )
    
config.voxel_size=0.03
config.target_fps=20
config.min_std=0.5

In [22]:
sequence_ts = fread.get_timstamps(config.get_feature_dir(), ext=".secondary.npz")

groundtruth_data = np.load(os.path.join(config.get_groundtruth_dir(), f"{config.get_file_name()}.pose.npz"))
groundtruth_t = groundtruth_data["trajectory_t"]

num_frames = len(sequence_ts)

local_pcds = []

for t in tqdm.trange(num_frames):
    feature_file = os.path.join(config.get_feature_dir(), f"{sequence_ts[t]}.secondary.npz")
    pcd = FCGF.get_features(feature_file, config.voxel_size, pcd_only=True)
    local_pcds.append(pcd)

100%|██████████| 407/407 [00:02<00:00, 138.73it/s]


In [28]:
fragments_gt = []

for t in range(0, num_frames, 10):
    local_temp = copy.deepcopy(local_pcds[t])
    local_temp.transform(groundtruth_t[t])
    fragments_gt.append(local_temp)
    
    if t == 0:
        trajectory = pointcloud.merge_pcds(fragments_gt, 0.03)

        points = np.asarray(trajectory.points)
        gx, gy = points[:, 0], points[:, 2]
        continue
    
    cpoints = np.asarray(local_temp.points)
    cx, cy = cpoints[:, 0], cpoints[:, 2]
    
    plt.figure(figsize=(5, 5))

    plt.scatter(gx, gy, s=0.1, c="#27ae60")
    plt.scatter(cx, cy, s=0.1, c="#e74c3c")
    # plt.xlim(-4.5, 4)
    # plt.ylim(-5.5, 4.5)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.axis("off")
    
    plt.savefig(f"results/presentation/slam/{t:03d}.jpeg")
    plt.close()
    
    trajectory = pointcloud.merge_pcds(fragments_gt, 0.03)

    points = np.asarray(trajectory.points)
    gx, gy = points[:, 0], points[:, 2]

In [31]:
img_files = glob.glob("results/presentation/slam/*.jpeg")
img_files = sorted(img_files, key=lambda x: int(os.path.basename(x).split(".")[0]))
images = []

for t in range(len(img_files)):
    img = cv2.imread(img_files[t])
    images.append(img)

video = cv2.VideoWriter(f"results/presentation/slam/video.avi", cv2.VideoWriter_fourcc(*'XVID'), 10, (360, 360))
for image in tqdm.tqdm(images):
    video.write(image)

video.release()

100%|██████████| 40/40 [00:00<00:00, 1412.16it/s]
